## Research on the reliability of borrowers

The customer is the bank's credit department. It is necessary to understand whether the marital status and the number of children of the client affects the fact of repayment of the loan on time. Input data from the bank — statistics on the solvency of customers.

The results of the study will be taken into account when building a model of * * credit scoring** — a special system that evaluates the ability of a potential borrower to repay a loan to the bank.

The purpose of the study is to analyze the influence of various factors on the fact of repayment of the loan on time.

The study is divided into 3 parts
* Part One-Learning General Information
  * Learning files, getting information, downloading libraries
* Part Two-Data Preprocessing
  * Processing omissions
  * Replacing the data type
  * Handling duplicates
  * Lemmatization and categorization of data
* Part Three-Analysis of the received data
  * Analysis of the correlation between the fact of repayment of the loan and various factors
    

# Step 1. Study the general information.

## Studying files, getting information, loading libraries.

In [1]:
import pandas as pd

In [2]:
!pip install pymystem3

Defaulting to user installation because normal site-packages is not writeable


In [3]:
from pymystem3 import Mystem
m = Mystem()

In [4]:
df = pd.read_csv('/datasets/data.csv')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [6]:
print(df.isnull().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


### Conclusion

Looking at the information on the dataframe, you can understand that the omissions occur in two columns:total_income and days_employed. It is too early to name the possible reasons, but I would venture to suggest that it is often difficult for people to name the exact number of working days, and the reason may also be that people want to hide their real work experience. As for total_income, the reasons may be the same: ignorance or unwillingness to disclose the truth about earnings.

# Step 2. Data Preprocessing

## Processing omissions


Since the values of both columns are of the Float64 type, we will replace the gaps with the median or average.
there are a lot of values, and therefore it is difficult to say for sure that using the replacement of the arithmetic mean, we will not spoil our sample, moreover, among the values there are incredible values, translating into years that are more than 1000 years, so I decided to use the median for this purpose

In [7]:
print(df['days_employed'].mean(), '- среднее для столбца days_employed')
print(df['days_employed'].median(), '- медиана для столбца days_employed')
print(df['total_income'].mean(), '- среднее для столбца total_income')
print(df['total_income'].median(), '- медиана для столбца total_income')

63046.49766147338 - среднее для столбца days_employed
-1203.369528770489 - медиана для столбца days_employed
167422.30220817294 - среднее для столбца total_income
145017.93753253992 - медиана для столбца total_income


In [8]:
df['days_employed'] = df['days_employed'].fillna(df['days_employed'].median())

Days at work can, in my opinion, be replaced by the median

As for the total income, then, having picked up the idea in the general chat, I decided to assign empty values not just median values, but corresponding to the total_income column, so that the data would be the least distorted

In [9]:
medians_total_income = df.groupby('income_type')['total_income'].median()

найдем медианы для дохода для каждого типа занятости, а затем заменим значения

In [10]:
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'сотрудник'), 'total_income'] = medians_total_income[6]
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'компаньон'), 'total_income'] = medians_total_income[3]
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'пенсионер'), 'total_income'] = medians_total_income[4]
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'госслужащий'), 'total_income'] = medians_total_income[2]
df.loc[(df['total_income'].isnull()) & (df['income_type'] == 'предприниматель'), 'total_income'] = medians_total_income[5]

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Помимо этого, в данных встречаются странные нулевые значения возраста клиентов, посмотрим, как они распределены в зависимости от занятости

In [12]:
df.loc[df['dob_years'] == 0, 'income_type'].value_counts()

сотрудник      55
пенсионер      20
компаньон      20
госслужащий     6
Name: income_type, dtype: int64

Теперь мы проделаем то же самое, что и с пропусками значений, однако по отношению к возрасту мы можем применить среднее арифметическое значение для замены, а не медиану

In [13]:
age_means = df.groupby('income_type')['dob_years'].mean()

In [14]:
age_means

income_type
безработный        38.000000
в декрете          39.000000
госслужащий        40.636737
компаньон          39.697542
пенсионер          59.063019
предприниматель    42.500000
сотрудник          39.821027
студент            22.000000
Name: dob_years, dtype: float64

In [15]:
df.loc[(df['dob_years'] == 0) & (df['income_type'] == 'сотрудник'), 'dob_years'] = age_means[6]
df.loc[(df['dob_years'] == 0) & (df['income_type'] == 'пенсионер'), 'dob_years'] = age_means[4]
df.loc[(df['dob_years'] == 0) & (df['income_type'] == 'компаньон'), 'dob_years'] = age_means[3]
df.loc[(df['dob_years'] == 0) & (df['income_type'] == 'госслужащий'), 'dob_years'] = age_means[2]

In [16]:
df.loc[df['dob_years'] == 0, 'dob_years'].value_counts()

Series([], Name: dob_years, dtype: int64)

Нулевых значений для возраста не осталось

In [17]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

тут есть ошибочные значения, скорее всего -1 значит 1 ребенок, а 20 - 2 ребенка

In [18]:
df.loc[df['children'] == 20, 'children'] = 2
df.loc[df['children'] == -1, 'children'] = 1

In [19]:
df['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

### Вывод

Помимо пропущенных значений, я заменил некоторые артефакты в значениях, чтобы данные были более пригодными для последующей обработки

## Замена типа данных

На мой взгляд, единственные столбцы, где тип данных несоотвественный - это total_income и days_employed, их я и поменяю этот метод я выбрал, потому что он на всякий случай также все ошибки заменяет на NaN, чтобы проверить в очередной раз, не осталось ли артефактов

In [20]:
df['days_employed'] = pd.to_numeric(df['days_employed'], errors='coerce').astype('int')
df['total_income'] = pd.to_numeric(df['total_income'], errors='coerce').astype('int')

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null float64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


### Вывод

Тип данных также был изменен, для того чтобы в дальнейшем использовать эти данные для анализа 

## Обработка дубликатов

Настало время дубликатов, удалять будем только то, где полностью совпадают строки, заодно скинем индексы
с большой долей вероятности, причиной возникновения дубликтов стали ошибки заполнения, ошибки в программе и так далее

In [22]:
print(df.duplicated().sum(), '- количество дубликатов')
print()
df = df.drop_duplicates().reset_index(drop=True)
print(df.duplicated().sum(), '- количество дубликатов после удаления')

54 - количество дубликатов

0 - количество дубликатов после удаления


Помимо этого, в столбце образование присутсвует много записей, идентичных друг другу, но исполненных в разном регистре

In [23]:
df['education'].value_counts()

среднее                13705
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

Приведем все значения к единому нижнему регистру

In [24]:
df['education'] = df['education'].str.lower()

In [25]:
df['education'].value_counts()

среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

### Вывод

Здесь были убраны дубликаты, произведена замена значений в образовании

## Лемматизация и категоризация данных

Посмотрим, сколько в датафрейме вариантов целей кредитования

In [26]:
df['purpose'].value_counts()

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
покупка жилья для сдачи                   652
операции с жильем                         652
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Выделим 4 основных причины кредитования и с помощью функции лемматизации запишем значения в новый столбец

Теперь нужно провести лемматизацию, а так как у нас уже есть основная группа причин кредитования, то запишем ее в новый столбец, если лемма совпадает с причиной

In [27]:
def purpose_change(row):
    purpose = " ".join(m.lemmatize(row['purpose']))
    if 'автом' in purpose:
        return 'автомобиль'
    elif 'свадь' in purpose:
        return  'свадьба'
    elif 'образов' in purpose:
        return 'образование'
    elif 'недвиж' in purpose or 'жиль' in purpose:
        return 'недвижимость'
    else:
        return row['purpose']   

In [28]:
df['purpose_new']= df.apply(purpose_change, axis = 1)     

In [29]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_new
0,1,-8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,-4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,-5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,-4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


### Вывод

Были ввыделены 4 категории: недвижимость, автомобиль, образование, свадьба. На основании этих категорий была произведена лемматизация и появился новый столбец, с более общими причинами.

Первое, что мы должны категоризировать, после того как уже была добавлена новая категория с целями кредитования, на мой взгляд - это возраст.

In [30]:
age_statistics = df['dob_years'].describe()

In [31]:
age_statistics

count    21471.000000
mean        43.484430
std         12.228484
min         19.000000
25%         33.500000
50%         43.000000
75%         53.000000
max         75.000000
Name: dob_years, dtype: float64

Минимальное значение в нашем случае составляет 19 лет, так как все нули мы убрали и заменили на медианы, поэтому категоризировать в соответствии с делением: подросток, взрослый, пенсионер - не получится

Можно разделить категории по квантилям: до 34, 34-43, 44-53, от 53

In [32]:
def age_group(age):
    if age <= age_statistics[4]:
        return '>34'
    elif age_statistics[4] < age <= age_statistics[5]:
        return '34-43'
    elif age_statistics[5] < age <= age_statistics[6]:
        return '43-53'
    else: 
        return '53+'
    

In [33]:
df['age_group'] = df['dob_years'].apply(age_group)

In [34]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_new,age_group
0,1,-8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,34-43
1,1,-4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,34-43
2,0,-5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,>34
3,3,-4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,>34
4,0,340266,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,43-53


Теперь можно категоризировать похожим способом и по доходам

In [35]:
statistics = df['total_income'].describe()

In [36]:
statistics

count    2.147100e+04
mean     1.652954e+05
std      9.815346e+04
min      2.066700e+04
25%      1.076545e+05
50%      1.425940e+05
75%      1.957675e+05
max      2.265604e+06
Name: total_income, dtype: float64

1 группа характеризуется доходом до 107654, 2 - от 107654 до 142594, 3 - от 142594 до 195767, 4 от 195767

In [37]:
def determine_income_group(income):
    if income <= statistics[4]:
        return 1
    elif statistics[4] < income <= statistics[5]:
        return 2
    elif statistics[5] < income <= statistics[6]:
        return 3
    else: 
        return 4

In [38]:
df['income_group'] = df['total_income'].apply(determine_income_group)

In [39]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_new,age_group,income_group
0,1,-8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,34-43,4
1,1,-4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,34-43,2
2,0,-5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,>34,3
3,3,-4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,>34,4
4,0,340266,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,43-53,3


Добавим еще один столбец, который будет показывать, если и дети у потенциального клиента, через специальную программу

In [40]:
def determine_children(children):
    if children > 0:
        return 'есть дети'
    else:
        return 'нет детей'

In [41]:
df['children_status'] = df['children'].apply(determine_children)

In [42]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_new,age_group,income_group,children_status
0,1,-8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,34-43,4,есть дети
1,1,-4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,34-43,2,есть дети
2,0,-5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,>34,3,нет детей
3,3,-4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,>34,4,есть дети
4,0,340266,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,43-53,3,нет детей


Посмотрим статистику по всем категориям, выделенными мной

In [43]:
df['purpose_new'].value_counts()

недвижимость    10814
автомобиль       4308
образование      4014
свадьба          2335
Name: purpose_new, dtype: int64

In [44]:
df['age_group'].value_counts()

34-43    5878
>34      5368
53+      5284
43-53    4941
Name: age_group, dtype: int64

In [45]:
df['income_group'].value_counts()

2    5490
4    5368
1    5368
3    5245
Name: income_group, dtype: int64

In [46]:
df['children_status'].value_counts()

нет детей    14107
есть дети     7364
Name: children_status, dtype: int64

### Вывод

Самая популярная цель кредитования - недвижимость, самый часто встречаемый возраст клиента - 34-43, а вторая группа дохода(от 107654 до 142594) - самая популярная, а детей нет у большего числа потенциальных кредитованных

# Шаг 3. Анализ полученных данных

Первое, что предстоит - выяснить, есть ли зависимость между наличием детей и возвратом кредита в срок?

добавим еще один столбец, который будет показывать, если и дети у потенциального клиента, через специальную программу

In [47]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_new,age_group,income_group,children_status
0,1,-8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,34-43,4,есть дети
1,1,-4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,34-43,2,есть дети
2,0,-5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,>34,3,нет детей
3,3,-4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,>34,4,есть дети
4,0,340266,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,43-53,3,нет детей


Для ответа на вопрос можем построить сводную таблицу

In [48]:
df_pivot = df.pivot_table(index = ['children_status'], values = 'debt')

In [49]:
df_pivot.sort_values('debt', ascending=False)

,debt
children_status,
есть дети,0.092070
нет детей,0.075353


In [50]:
df_pivot1 = df.pivot_table(index=['children_status'], values='debt', aggfunc='sum')

In [51]:
df_pivot1

,debt
children_status,
есть дети,678
нет детей,1063


In [52]:
df_pivot['amount_debt'] = df_pivot1['debt']

In [53]:
df_pivot

,debt,amount_debt
children_status,,
есть дети,0.092070,678
нет детей,0.075353,1063


In [54]:
df['children_status'].value_counts().sort_values(ascending=True)

есть дети     7364
нет детей    14107
Name: children_status, dtype: int64

In [55]:
df_pivot['amount'] = df['children_status'].value_counts().sort_values(ascending=True)

In [56]:
df_pivot

,debt,amount_debt,amount
children_status,,,
есть дети,0.092070,678,7364
нет детей,0.075353,1063,14107


Вот здесь у меня появились некоторые проблемы, потому что я не знаю, как уместить код построения сводной таблицы в одну строчку, поэтому сделал как мог, присоединяя поочередно каждый столбец.

### Вывод

Данная сводная таблица показывает, что наличие детей увеличивает шансы по задолженности на 1,7%. Это говорит о том, что кредитуемый предпочтет не платить деньги банку, чем не обеспечить нужным своего ребенка или детей

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [57]:
df['family_status'].value_counts()

женат / замужем          12344
гражданский брак          4163
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

Можем также построить сводную таблицу, для ответа на поставленный вопрос

In [58]:
df_pivot2 = df.pivot_table(index=['family_status'], values='debt')

In [59]:
df_pivot2.sort_values('debt', ascending=False)

,debt
family_status,
Не женат / не замужем,0.097509
гражданский брак,0.093202
женат / замужем,0.075421
в разводе,0.071130
вдовец / вдова,0.065693


In [60]:
df_pivot3 = df.pivot_table(index=['family_status'], values='debt', aggfunc='sum')

In [61]:
df_pivot3

,debt
family_status,
Не женат / не замужем,274
в разводе,85
вдовец / вдова,63
гражданский брак,388
женат / замужем,931


In [62]:
df_pivot2['amount_debt'] = df_pivot3['debt']

In [63]:
df_pivot2

,debt,amount_debt
family_status,,
Не женат / не замужем,0.097509,274
в разводе,0.071130,85
вдовец / вдова,0.065693,63
гражданский брак,0.093202,388
женат / замужем,0.075421,931


In [64]:
df['family_status'].value_counts().sort_values(ascending=True)

вдовец / вдова             959
в разводе                 1195
Не женат / не замужем     2810
гражданский брак          4163
женат / замужем          12344
Name: family_status, dtype: int64

In [65]:
df_pivot2['amount'] = df['family_status'].value_counts().sort_values(ascending=True)

In [66]:
df_pivot2

,debt,amount_debt,amount
family_status,,,
Не женат / не замужем,0.097509,274,2810
в разводе,0.071130,85,1195
вдовец / вдова,0.065693,63,959
гражданский брак,0.093202,388,4163
женат / замужем,0.075421,931,12344


### Вывод

Данная сводная таблица говорит нам о том, что люди находящиеся в браке, в разводе и вдовцы имеют наибольший шанс вернуть долг банку, в то время как холостые и люди в гражданском браке наименьший, что связано, наверняка, с отсуствием гарантированной финансовой поддержки со стороны близкого человека, елси мы сравниваем с женатыми

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Также воспользуемся сводной таблицей

In [67]:
df_pivot4 = df.pivot_table(index=['income_group'], values='debt')

In [68]:
df_pivot4.sort_values('debt', ascending=False)

,debt
income_group,
2,0.087978
3,0.085415
1,0.079545
4,0.071349


Напомню - 1 группа характеризуется доходом до 107654, 2 - от 107654 до 142594, 3 - от 142594 до 195767, 4 от 195767

In [69]:
df_pivot5 = df.pivot_table(index=['income_group'], values='debt', aggfunc='sum')

In [70]:
df_pivot5

,debt
income_group,
1,427
2,483
3,448
4,383


In [71]:
df_pivot4['amount_debt'] = df_pivot5['debt']

In [72]:
df_pivot4

,debt,amount_debt
income_group,,
1,0.079545,427
2,0.087978,483
3,0.085415,448
4,0.071349,383


In [73]:
df['income_group'].value_counts().sort_values(ascending=True)

3    5245
4    5368
1    5368
2    5490
Name: income_group, dtype: int64

In [74]:
df_pivot4['amount'] = df['income_group'].value_counts().sort_values(ascending=True)

In [75]:
df_pivot4

,debt,amount_debt,amount
income_group,,,
1,0.079545,427,5368
2,0.087978,483,5490
3,0.085415,448,5245
4,0.071349,383,5368


### Вывод

Что удивительно, но именно вторая и третья группа имеют наименьшие шансы вернуть долг, хотя положение в середине должно означать также и наибольшую стабильность, возможно, это также связано и с тем, что люди из этих групп худшим образом распределяются своими деньгами. Ожидаемо, люди с самым высоким доходом, возвращают долги лучше всех остальных.

- Как разные цели кредита влияют на его возврат в срок?

In [76]:
df_pivot6 = df.pivot_table(index=['purpose_new'], values='debt')

In [77]:
df_pivot6.sort_values('debt', ascending=False)

,debt
purpose_new,
автомобиль,0.093547
образование,0.092177
свадьба,0.079657
недвижимость,0.072314


In [78]:
df_pivot7 = df.pivot_table(index=['purpose_new'], values='debt', aggfunc='sum')

In [79]:
df_pivot7

,debt
purpose_new,
автомобиль,403
недвижимость,782
образование,370
свадьба,186


In [80]:
df_pivot6['amount_debt'] = df_pivot7['debt']

In [81]:
df_pivot6

,debt,amount_debt
purpose_new,,
автомобиль,0.093547,403
недвижимость,0.072314,782
образование,0.092177,370
свадьба,0.079657,186


In [82]:
df['purpose_new'].value_counts().sort_values(ascending=True)

свадьба          2335
образование      4014
автомобиль       4308
недвижимость    10814
Name: purpose_new, dtype: int64

In [83]:
df_pivot6['amount'] = df['purpose_new'].value_counts().sort_values(ascending=True)

In [84]:
df_pivot6

,debt,amount_debt,amount
purpose_new,,,
автомобиль,0.093547,403,4308
недвижимость,0.072314,782,10814
образование,0.092177,370,4014
свадьба,0.079657,186,2335


### Вывод

Из таблицы видно, что лучши всего возращают долги те, кто берет кредит для покупки недвижимости. Данный тип прриобретения наиболее масштабный, а потому к нему подходят наиболее отвественным способом. В то время как покупка машины в кредит яявляется достаточно высоким гарантом невозврата долга, скорее всего это связано с тем, что очень много молодых людей берет кредиты на машины.

## Step 4. General output

Conclusion: the ideal client of the bank is a person who is married or divorced, without children, whose goals are to buy real estate, his income is from 195767. This characterizes him as a person who knows how to manage money, selfish enough not to have children, and consequently not to have additional expenses, but at the same time responsible enough, because he is already married or was in it.